In [13]:
import datafed.CommandLib
import getpass
import datafed.MessageLib

# Initialize the API object
api = datafed.CommandLib.API()
mapi = datafed.MessageLib.API()


def DataFed_Log_In():
    mapi.logout()
    # Prompt for user ID and password
    uid = input("User ID: ")
    password = getpass.getpass(prompt="Password: ")

    try:
        # Attempt to log in using provided credentials
        mapi.manualAuthByPassword(uid, password)
        success = f"Successfully logged in to Data as {api.getAuthUser()}"
    except:
        success = "Could not log into DataFed. Check your internet connection, username, and password"

    return success

Exception: Server host is not defined

In [22]:
import os

directory = r"C:\Users\Asylum User\Documents\Asylum Research Data\230802"
filename = "Image0006.ibw"
file_path = os.path.join(directory, filename)

if os.path.exists(file_path):
    print("File found:", file_path)
else:
    print("File not found:", file_path)


File found: C:\Users\Asylum User\Documents\Asylum Research Data\230802\Image0006.ibw


In [8]:
# Call the login function and print the resul
login_result = DataFed_Log_In()
print(login_result)


Successfully logged in to Data as u/yig319


In [ ]:
import argparse
from util import *
from datafed.CommandLib import API
import os
import glob

def _send_ibw_to_datafed(file_name, collection_id):
    df_api = API()
    
    json_output = get_metadata(file_name)

    print(file_name)
    print(collection_id)

    # This removes flattening information and fixes inf values in metadata
    try: 
        del json_output['Flatten Offsets 0']
    except:
        pass

    try:
        del json_output['Flatten Slopes 0']
    except:
        pass

    try:
        del json_output['Flatten Slopes 4']
    except:
        pass

    try: 
        del json_output['Flatten Offsets 4']
    except:
        pass

    try: 
        del json_output['Flatten Offsets 1']
    except:
        pass

    try:
        del json_output['Flatten Slopes 1']
    except:
        pass

    for i, (key, value) in enumerate(json_output.items()):
        if value == np.NINF:
            json_output[key] = '-Inf'

    for i, (key, value) in enumerate(json_output.items()):
        if value == np.Inf:
            json_output[key] = 'Inf'

    try:
        # creates a new data record
        dc_resp = df_api.dataCreate(os.path.basename(file_name), # file name
                                metadata=json.dumps(json_output), # metadata
                                parent_id=collection_id, # parent collection
                            )
    except Exception:

        print('There was an error creating the DataRecord')

    try: 
        # extracts the record ID
        rec_id = dc_resp[0].data[0].id
    except ValueError:
        print('Could not find record ID')

    try:
        # sends the put command
        put_resp = df_api.dataPut(rec_id,
                                    file_name,
                                    wait = True)
    except Exception:
        print('Could not intiate globus transfer')

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="This is a function that sends ibw files from Oxford Instruments Asylum Research Atomic Force Microscopes to DataFed"
    )
    parser.add_argument("file_name", help="This is the name of the full file path")
    parser.add_argument("collection_id", help="This is the name of the collection ID where the file will be saved on DataFed")
    args = parser.parse_args()

    if os.path.isdir(args.file_name):

        file_name = r'C:\Users\Asylum User\Documents\Asylum Research Data\230802\Image0006.ibw'

        if os.path.isdir(args.file_name):
            file_list = glob.glob(args.file_name + "\**\*.ibw", recursive= True)

            for file in file_list:
                try:
                    _send_ibw_to_datafed(file, args.collection_id)
                except:
                    pass
        
        else:
            _send_ibw_to_datafed(args.file_name, args.collection_id) 